# Detailed Exploration of Universal Gate Set for Intermediate Zeta

## Header

In [ ]:
import numpy as np
import QDYN
from QDYN.shutil import mkdir
from QDYN.pulse import Pulse, pulse_tgrid
from QDYN.gate2q import Gate2Q
from QDYN.linalg import norm, vectorize
from os.path import join
from notebook_utils import (
    get_zeta_table, plot_zeta_data, filter_table, get_logical_energies_table,
    get_stage4_table, bare_decomposition, get_transition_table, J_target)
from mgplottools.io import writetotxt
from select_for_stage4 import GATE_RE, GATE_IM, GATE
from generate_zeta_universal import write_config
from analytical_pulses import AnalyticalPulse
from run_oct import propagate
from collections import OrderedDict
from zeta_systematic_variation import systematic_variation
import pandas as pd

In [111]:
H_left = GATE['H_left']
H_right = GATE['H_right']
def write_target_gate_dat(rf, gate):
    writetotxt(join(rf, 'target_gate.dat'), GATE_RE[gate], GATE_IM[gate])

In [112]:
def J_PE(U):
    if U is None:
        return None
    else:
        C = U.closest_unitary().concurrence()
        max_loss = np.max(1.0 - U.logical_pops())
        return J_target('PE', C, max_loss)

def F_avg_H_left(U):
    if U is None:
        return None
    else:
        return U.F_avg(H_left)
    
def F_avg_H_right(U):
    if U is None:
        return None
    else:
        return U.F_avg(H_right)

In [ ]:
from multiprocessing import Process
import os
def system(cmd):
    p = Process(target=os.system, args=(cmd,))
    p.start()
    return p

In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [ ]:
get_zeta_table = QDYN.memoize.memoize(get_zeta_table)
get_zeta_table.load('zeta_table.cache')

In [ ]:
zeta_table = get_zeta_table('./runs_050_RWA', T=50)

In [ ]:
w_max = 1.0 # minimum frequency in the rotating frame to be resolved. (for calculation of nt)

In [ ]:
n_qubit = 5; n_cavity = 6

## Analysis

### Properties of the Hamiltonian

In [113]:
w1 = 6.00 # GHz
w2 = 6.32 # GHz
wc = 5.75 # GHZ
root = './runs_zeta_detailed/w2_%dMHz_wc_%dMHz' % (w2*1000, wc*1000)
mkdir(root)

* what is the the entangling energy?

In [114]:
zeta = float(filter_table(zeta_table, [(w2, wc),])['zeta [MHz]']); zeta

-16.716000000000008

In [115]:
T_entangling = 500.0 / abs(zeta); T_entangling # ns

29.911462072266076

####  PE

* Verify the PE. What do the logical eigenstates look like? What are the dressed qubit levels? What required spectral resolution do we anticipate?

In [116]:
T = T_entangling
rf = join(root, 'verify_PE'); mkdir(rf)
nt = int(max(2000, 100 * w_max * T))
write_config(join(rf, 'config'), T, nt, wc, w2, wd=6.0, gate="CPHASE", J_T='SM', prop_guess='T')
pulse = Pulse(tgrid=pulse_tgrid(T, nt), time_unit='ns',ampl_unit='MHz')  # zero amplitude
pulse.write(join(rf, 'pulse.guess'))
U = propagate(rf, 'pulse.guess', rwa=True, keep=True, force=True)
print "concurrence: %.5f" % U.concurrence()

concurrence: 1.00000


In [117]:
psi = {}; psi['00'], psi['01'], psi['10'], psi['11'] \
= np.genfromtxt(join(rf, 'logical_states.dat'), unpack=True, usecols=range(4))
for label in ['00', '01', '10', '11']:
    print("%s = %s" % (label, bare_decomposition(psi[label], n_qubit, n_cavity)))

00 = 100.0% {000}
01 = 1.5% {001} + 98.4% {010}
10 = 5.7% {001} + 0.3% {010} + 94.0% {100}
11 = 33.1% {020} + 1.1% {101} + 65.8% {110}


In [118]:
transition_table = get_transition_table(filter_table(zeta_table, [(w2, wc),]))
transition_table.reset_index(drop=True)

,E01 - E00 [MHz],E10 - E00 [MHz],E11 - E10 [MHz],E11 - E01 [MHz],bare w2 [MHz],bare w1 [MHz]
0,6328.69,6017.286,6311.974,6000.57,6320,6000


In [119]:
w1_dressed0 = transition_table['E10 - E00 [MHz]'][0] / 1000.0 # GHz
w1_dressed1 = transition_table['E11 - E01 [MHz]'][0] / 1000.0 # GHz
w2_dressed0 = transition_table['E01 - E00 [MHz]'][0] / 1000.0 # GHz
w2_dressed1 = transition_table['E11 - E10 [MHz]'][0] / 1000.0 # GHz

In [120]:
print("Separation of dressed frequencies, left qubit: %.3f MHz"
      % abs(float(transition_table['E11 - E01 [MHz]']) - float(transition_table['E10 - E00 [MHz]'])))
print("Separation of dressed frequencies, right qubit: %.3f MHz"
      % abs(float(transition_table['E11 - E10 [MHz]']) - float(transition_table['E01 - E00 [MHz]'])))

Separation of dressed frequencies, left qubit: 16.716 MHz
Separation of dressed frequencies, right qubit: 16.716 MHz


In [121]:
print "Spectral resolution of pulse: %.1f MHz" % pulse.dw(freq_unit='MHz')

Spectral resolution of pulse: 33.4 MHz


####  SQ

* Verify the local gate

In [122]:
T = 2*T_entangling
rf = join(root, 'verify_SQ'); mkdir(rf)
nt = int(max(2000, 100 * w_max * T))
write_config(join(rf, 'config'), T, nt, wc, w2, wd=6.0, gate="CPHASE", J_T='SM', prop_guess='T')
pulse = Pulse(tgrid=pulse_tgrid(T, nt), time_unit='ns',ampl_unit='MHz')  # zero amplitude
pulse.write(join(rf, 'pulse.guess'))
print "Spectral resolution of pulse: %.1f MHz" % pulse.dw(freq_unit='MHz')
U = propagate(rf, 'pulse.guess', rwa=True, keep=True, force=True)
print "concurrence: %.5f" % U.concurrence()

Spectral resolution of pulse: 16.7 MHz
concurrence: 0.00007


In [123]:
T = 4*T_entangling
rf = join(root, 'verify_SQ_2cycles'); mkdir(rf)
nt = int(max(2000, 100 * w_max * T))
write_config(join(rf, 'config'), T, nt, wc, w2, wd=6.0, gate="CPHASE", J_T='SM', prop_guess='T')
pulse = Pulse(tgrid=pulse_tgrid(T, nt), time_unit='ns',ampl_unit='MHz')  # zero amplitude
pulse.write(join(rf, 'pulse.guess'))
print "Spectral resolution of pulse: %.1f MHz" % pulse.dw(freq_unit='MHz')
U = propagate(rf, 'pulse.guess', rwa=True, keep=True, force=True)
print "concurrence: %.5f" % U.concurrence()

Spectral resolution of pulse: 8.4 MHz
concurrence: 0.00014


### Improving the PE

* Can we compress the PE by adding a pulse and OCT?

In [124]:
compression = 0.7
wd = 0.5*(w1+w2)
rf = join(root, 'compress_PE_%.1f' % compression); mkdir(rf)

In [125]:
rf

'./runs_zeta_detailed/w2_6320MHz_wc_5750MHz/compress_PE_0.7'

In [126]:
# write runfolder
T = compression*T_entangling
nt = int(max(2000, 100 * w_max * T))
write_config(join(rf, 'config'), T, nt, wc, w2, wd=wd, gate="CPHASE", J_T='PE', prop_guess='F')

In [127]:
pulse = AnalyticalPulse('1freq_rwa', T, nt,
            parameters={'E0': 0.0, 'T': T, 'w_L': wd, 'w_d': wd},
            t0=0.0, time_unit='ns', ampl_unit='MHz', freq_unit='MHz', mode="complex")
systematic_variation(rf, pulse, vary=OrderedDict([
            ('E0', [0.0, 50, 200, 500]), 
            ('w_L', [w1, w2, 0.5*(w1+w2),  
                     w1_dressed0, w1_dressed1, 0.5*(w1_dressed0+w1_dressed1),
                     w2_dressed0, w2_dressed1, 0.5*(w2_dressed0+w2_dressed1)])]),
            fig_of_merit=J_PE, n_procs=40)

,E0,w_L,fig_of_merit
20,200,6.160000,0.065954
11,50,6.160000,0.108181
0,0,6.000000,0.111862
1,0,6.320000,0.111862
2,0,6.160000,0.111862
3,0,6.017286,0.111862
4,0,6.000570,0.111862
5,0,6.008928,0.111862
6,0,6.328690,0.111862
7,0,6.311974,0.111862


In [128]:
pulse = AnalyticalPulse('1freq_rwa', T, nt,
            parameters={'E0': 200.0, 'T': T, 'w_L': 6.16, 'w_d': 6.16},
            t0=0.0, time_unit='ns', ampl_unit='MHz', freq_unit='MHz', mode="complex")
pulse.write(join(rf, 'pulse_guess.json'))

In [129]:
U_guess = propagate(rf, 'pulse_guess.json', rwa=True, force=True, keep=None)
print("Concurrence: %f" % U_guess.closest_unitary().concurrence())
print("Loss: %f" % U_guess.pop_loss())

Concurrence: 0.944892
Loss: 0.005964


In [130]:
proc = system("./run_oct.py --target=PE --rwa --pre-simplex='pulse_guess.json' {rf}".format(rf=rf))

In [135]:
proc.join(); U = propagate(rf, 'pulse.dat', rwa=True, force=True);
! tail {rf}/oct_iters.dat | cut -b 1-80

  878  2.441503128017E-03  1.021673162694E-07  0.000000000000E+00  2.44160529533
  879  2.441275218414E-03  1.019170565411E-07  0.000000000000E+00  2.44137713547
  880  2.441047888456E-03  1.016675715323E-07  0.000000000000E+00  2.44114955602
  881  2.440821136252E-03  1.014188581838E-07  0.000000000000E+00  2.44092255511
  882  2.440594960006E-03  1.011709134452E-07  0.000000000000E+00  2.44069613091
  883  2.440369357867E-03  1.009237342714E-07  0.000000000000E+00  2.44047028160
  884  2.440144328032E-03  1.006773176279E-07  0.000000000000E+00  2.44024500535
  885  2.439919868680E-03  1.004316604860E-07  0.000000000000E+00  2.44002030034
  886  2.439695978011E-03  1.001867598248E-07  0.000000000000E+00  2.43979616477
  887  2.439472654232E-03  9.994261263395E-08  0.000000000000E+00  2.43957259684


In [136]:
U.concurrence() - U_guess.concurrence()

0.011012108861070957

In [137]:
U_closest_PE = U.closest_PE()

In [138]:
U.F_avg(U_closest_PE)

0.9890368481758246

### Implementation of Single-Qubit Gate

#### $H_L$ at $T = 2 T_\pi$ ####

In [139]:
rf = join(root, '1cycle_H_left'); mkdir(rf)
wd = 0.5*(w1+w2)

In [140]:
rf

'./runs_zeta_detailed/w2_6320MHz_wc_5750MHz/1cycle_H_left'

In [92]:
T = 2*T_entangling
nt = int(max(2000, 100 * w_max * T))
write_target_gate_dat(rf, 'H_left')
write_config(join(rf, 'config'), T, nt, wc, w2, wd=wd, gate="target_gate.dat", J_T='SM', prop_guess='F')

In [93]:
pulse = AnalyticalPulse('1freq_rwa', T, nt,
            parameters={'E0': 0.0, 'T': T, 'w_L': wd, 'w_d': wd},
            t0=0.0, time_unit='ns', ampl_unit='MHz', freq_unit='MHz', mode="complex")
systematic_variation(rf, pulse, vary=OrderedDict([
            ('E0', [0.0, 50, 100, 150]), 
            ('w_L', [w1, w2, 0.5*(w1+w2),  
                     w1_dressed0, w1_dressed1, 0.5*(w1_dressed0+w1_dressed1),
                     w2_dressed0, w2_dressed1, 0.5*(w2_dressed0+w2_dressed1)])]),
            fig_of_merit=F_avg_H_left, n_procs=40)

,E0,w_L,fig_of_merit
27,150,6.000000,0.193467
31,150,6.000570,0.195021
15,50,6.328690,0.199310
4,0,6.000570,0.199350
0,0,6.000000,0.199350
1,0,6.320000,0.199350
2,0,6.160000,0.199350
3,0,6.017286,0.199350
5,0,6.008928,0.199350
6,0,6.328690,0.199350


In [94]:
pulse = AnalyticalPulse('1freq_rwa', T, nt,
            parameters={'E0': 100.0, 'T': T, 'w_L': 6.017286, 'w_d': 6.017286},
            t0=0.0, time_unit='ns', ampl_unit='MHz', freq_unit='MHz', mode="complex")
pulse.write(join(rf, 'pulse_guess.json'))

In [95]:
U_guess = propagate(rf, 'pulse_guess.json', rwa=True, force=True)
print("Concurrence: %f" % U_guess.closest_unitary().concurrence())
print("Loss: %f" % U_guess.pop_loss())

Concurrence: 0.939675
Loss: 0.028905


In [96]:
U_guess.F_avg(H_left)

0.48383223068241632

In [ ]:
proc2 = system("./run_oct.py --rwa --pre-simplex='pulse_guess.json' {rf}".format(rf=rf))

In [ ]:
proc2.join()

In [98]:
rf

'./runs_zeta_detailed/w2_6320MHz_wc_5750MHz/1cycle_H_left'

In [100]:
U = propagate(rf, 'pulse.dat', rwa=True, force=True);
! tail {rf}/oct_iters.dat | cut -b 1-80

 2380  1.027234049713E-02  1.014422339535E-07  0.000000000000E+00  1.02724419393
 2381  1.027212597375E-02  1.012737457218E-07  0.000000000000E+00  1.02722272474
 2382  1.027191180872E-02  1.011055950323E-07  0.000000000000E+00  1.02720129143
 2383  1.027169800128E-02  1.009377816437E-07  0.000000000000E+00  1.02717989390
 2384  1.027148455073E-02  1.007703053068E-07  0.000000000000E+00  1.02715853210
 2385  1.027127145633E-02  1.006031657705E-07  0.000000000000E+00  1.02713720594
 2386  1.027105871730E-02  1.004363627747E-07  0.000000000000E+00  1.02711591536
 2387  1.027084633293E-02  1.002698960581E-07  0.000000000000E+00  1.02709466028
 2388  1.027063430250E-02  1.001037653547E-07  0.000000000000E+00  1.02707344062
 2389  1.027042262523E-02  9.993797039167E-08  0.000000000000E+00  1.02705225632


In [101]:
U.concurrence() - U_guess.concurrence()

-0.93411181081219918

In [102]:
U.F_avg(H_left)

0.98973377033203036

####  $H_L$ at $T = 40$ ns

In [104]:
rf = join(root, '40ns_H_left'); mkdir(rf)
wd = 0.5*(w1+w2)

In [105]:
rf

'./runs_zeta_detailed/w2_6320MHz_wc_5750MHz/40ns_H_left'

In [107]:
T = 40
nt = int(max(2000, 100 * w_max * T))
write_target_gate_dat(rf, 'H_left')
write_config(join(rf, 'config'), T, nt, wc, w2, wd=wd, gate="target_gate.dat", J_T='SM', prop_guess='F')

In [108]:
pulse = AnalyticalPulse('1freq_rwa', T, nt,
            parameters={'E0': 0.0, 'T': T, 'w_L': wd, 'w_d': wd},
            t0=0.0, time_unit='ns', ampl_unit='MHz', freq_unit='MHz', mode="complex")
systematic_variation(rf, pulse, vary=OrderedDict([
            ('E0', [0.0, 50, 150, 200]), 
            ('w_L', [w1, w2, 0.5*(w1+w2),  
                     w1_dressed0, w1_dressed1, 0.5*(w1_dressed0+w1_dressed1),
                     w2_dressed0, w2_dressed1, 0.5*(w2_dressed0+w2_dressed1)])]),
            fig_of_merit=F_avg_H_left, n_procs=40)

,E0,w_L,fig_of_merit
35,200,6.320332,0.199895
28,200,6.320000,0.199941
33,200,6.328690,0.200095
13,50,6.000570,0.205955
9,50,6.000000,0.206426
34,200,6.311974,0.207215
24,150,6.328690,0.213598
26,150,6.320332,0.214984
19,150,6.320000,0.215212
14,50,6.008928,0.218037


In [ ]:
pulse = AnalyticalPulse('1freq_rwa', T, nt,
            parameters={'E0': 200.0, 'T': T, 'w_L': 6.008928, 'w_d': 6.008928},
            t0=0.0, time_unit='ns', ampl_unit='MHz', freq_unit='MHz', mode="complex")
pulse.write(join(rf, 'pulse_guess.json'))

In [ ]:
U_guess = propagate(rf, 'pulse_guess.json', rwa=True, force=True)
print("Concurrence: %f" % U_guess.closest_unitary().concurrence())
print("Loss: %f" % U_guess.pop_loss())

In [ ]:
proc3 = system("./run_oct.py --rwa --pre-simplex='pulse_guess.json' {rf}".format(rf=rf))

In [103]:
proc3

<Process(Process-285, started)>

In [ ]:
proc3.join();

In [ ]:
U = propagate(rf, 'pulse.dat', rwa=True, force=True);
! tail {rf}/oct_iters.dat | cut -b 1-80

In [ ]:
U.concurrence() - U_guess.concurrence()

####  $H_R$ at $T = 40$ ns

In [141]:
rf = join(root, '40ns_H_right'); mkdir(rf)
wd = 0.5*(w1+w2)

In [142]:
rf

'./runs_zeta_detailed/w2_6320MHz_wc_5750MHz/40ns_H_right'

In [143]:
T = 40
nt = int(max(2000, 100 * w_max * T))
write_target_gate_dat(rf, 'H_right')
write_config(join(rf, 'config'), T, nt, wc, w2, wd=wd, gate="target_gate.dat", J_T='SM', prop_guess='F')

In [144]:
pulse = AnalyticalPulse('1freq_rwa', T, nt,
            parameters={'E0': 0.0, 'T': T, 'w_L': wd, 'w_d': wd},
            t0=0.0, time_unit='ns', ampl_unit='MHz', freq_unit='MHz', mode="complex")
systematic_variation(rf, pulse, vary=OrderedDict([
            ('E0', [0.0, 50, 150, 200]), 
            ('w_L', [w1, w2, 0.5*(w1+w2),  
                     w1_dressed0, w1_dressed1, 0.5*(w1_dressed0+w1_dressed1),
                     w2_dressed0, w2_dressed1, 0.5*(w2_dressed0+w2_dressed1)])]),
            fig_of_merit=F_avg_H_right, n_procs=40)

,E0,w_L,fig_of_merit
32,200,6.008928,0.191867
21,150,6.017286,0.193300
30,200,6.017286,0.193921
23,150,6.008928,0.215380
31,200,6.000570,0.223952
27,200,6.000000,0.226871
22,150,6.000570,0.255674
18,150,6.000000,0.258497
34,200,6.311974,0.295725
33,200,6.328690,0.310385


In [145]:
pulse = AnalyticalPulse('1freq_rwa', T, nt,
            parameters={'E0': 200.0, 'T': T, 'w_L': 6.160000, 'w_d': 6.160000},
            t0=0.0, time_unit='ns', ampl_unit='MHz', freq_unit='MHz', mode="complex")
pulse.write(join(rf, 'pulse_guess.json'))

In [146]:
U_guess = propagate(rf, 'pulse_guess.json', rwa=True, force=True)
print("Concurrence: %f" % U_guess.closest_unitary().concurrence())
print("Loss: %f" % U_guess.pop_loss())

Concurrence: 0.855990
Loss: 0.005123


In [147]:
U_guess.F_avg(H_right)

0.40103092217036956

In [148]:
proc4 = system("./run_oct.py --rwa --pre-simplex='pulse_guess.json' {rf}".format(rf=rf))

In [149]:
proc4

<Process(Process-567, started)>

In [ ]:
proc4.join();

In [ ]:
U = propagate(rf, 'pulse.dat', rwa=True, force=True);
! tail {rf}/oct_iters.dat | cut -b 1-80

In [ ]:
U.concurrence() - U_guess.concurrence()

In [ ]:
U.F_avg(H_right)